In [ ]:
#| default_exp auto

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
from os import cpu_count
import torch

from ray import tune
from ray.tune.search.basic_variant import BasicVariantGenerator

from neuralforecast.common._base_auto import BaseAuto
from neuralforecast.common._base_auto import MockTrial

from neuralforecast.models.rnn import RNN
from neuralforecast.models.gru import GRU
from neuralforecast.models.tcn import TCN
from neuralforecast.models.lstm import LSTM
from neuralforecast.models.deepar import DeepAR
from neuralforecast.models.dilated_rnn import DilatedRNN
from neuralforecast.models.bitcn import BiTCN

from neuralforecast.models.mlp import MLP
from neuralforecast.models.nbeats import NBEATS
from neuralforecast.models.nbeatsx import NBEATSx
from neuralforecast.models.nhits import NHITS
from neuralforecast.models.dlinear import DLinear
from neuralforecast.models.nlinear import NLinear
from neuralforecast.models.tide import TiDE
from neuralforecast.models.deepnpts import DeepNPTS

from neuralforecast.models.tft import TFT
from neuralforecast.models.vanillatransformer import VanillaTransformer
from neuralforecast.models.informer import Informer
from neuralforecast.models.autoformer import Autoformer
from neuralforecast.models.fedformer import FEDformer
from neuralforecast.models.patchtst import PatchTST
from neuralforecast.models.timesnet import TimesNet
from neuralforecast.models.itransformer import iTransformer

from neuralforecast.models.kan import KAN

from neuralforecast.models.stemgnn import StemGNN
from neuralforecast.models.hint import HINT
from neuralforecast.models.tsmixer import TSMixer
from neuralforecast.models.tsmixerx import TSMixerx
from neuralforecast.models.mlpmultivariate import MLPMultivariate
from neuralforecast.models.softs import SOFTS

from neuralforecast.losses.pytorch import MAE, MQLoss, DistributionLoss

In [ ]:
#| hide
import matplotlib.pyplot as plt

from fastcore.test import test_eq
from nbdev.showdoc import show_doc

import logging
import warnings
import inspect

from neuralforecast.losses.pytorch import MSE

In [ ]:
#| hide
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

plt.rcParams["axes.grid"]=True
plt.rcParams['font.family'] = 'serif'
plt.rcParams["figure.figsize"] = (6,4)

In [ ]:
#| hide
# Unit test to test that Auto* model contains all required arguments from BaseAuto class.

# Patch for Python 3.11 on get arg spec
if not hasattr(inspect, 'getargspec'):
    getargspec = inspect.getfullargspec
else:
    getargspec = inspect.getargspec

def test_args(auto_model, exclude_args=None):
    base_auto_args = getargspec(BaseAuto)[0]
    auto_model_args = getargspec(auto_model)[0]
    if exclude_args is not None:
        base_auto_args = [arg for arg in base_auto_args if arg not in exclude_args]
    args_diff = set(base_auto_args) - set(auto_model_args)
    assert not args_diff, f"__init__ of {auto_model.__name__} does not contain the following required variables from BaseAuto class:\n\t\t{args_diff}"

# Models

> NeuralForecast contains user-friendly implementations of neural forecasting models that allow for easy transition of computing capabilities (GPU/CPU), computation parallelization, and hyperparameter tuning.

All the NeuralForecast models are "global" because we train them with all the series from the input pd.DataFrame data `Y_df`, yet the optimization objective is, momentarily, "univariate" as it does not consider the interaction between the output predictions across time series. Like the StatsForecast library, `core.NeuralForecast` allows you to explore collections of models efficiently and contains functions for convenient wrangling of input and output pd.DataFrames predictions.

First we load the AirPassengers dataset such that you can run all the examples.

In [ ]:
%%capture
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengersDF as Y_df

In [ ]:
%%capture
# Split train/test and declare time series dataset
Y_train_df = Y_df[Y_df.ds<='1959-12-31'] # 132 train
Y_test_df = Y_df[Y_df.ds>'1959-12-31']   # 12 test
dataset, *_ = TimeSeriesDataset.from_df(Y_train_df)

# 1. Automatic Forecasting

## A. RNN-Based

In [ ]:
#| export
class AutoRNN(BaseAuto):
    
    default_config = {
        "input_size_multiplier": [-1, 4, 16, 64],
        "inference_input_size_multiplier": [-1],
        "h": None,
        "encoder_hidden_size": tune.choice([50, 100, 200, 300]),
        "encoder_n_layers": tune.randint(1, 4),
        "context_size": tune.choice([5, 10, 50]),
        "decoder_hidden_size": tune.choice([64, 128, 256, 512]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "max_steps": tune.choice([500, 1000]),
        "batch_size": tune.choice([16, 32]),
        "loss": None,
        "random_seed": tune.randint(1, 20)
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None,
                ):
        """ Auto RNN
        
        **Parameters:**<br>
        
        """
        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend)         

        super(AutoRNN, self).__init__(
              cls_model=RNN, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config, 
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series=None):
        config = cls.default_config.copy()
        config['input_size'] = tune.choice([h*x \
                        for x in config['input_size_multiplier']])
        config['inference_input_size'] = tune.choice([h*x \
                        for x in config['inference_input_size_multiplier']])
        del config['input_size_multiplier'], config['inference_input_size_multiplier']
        if backend == 'optuna':
            config = cls._ray_config_to_optuna(config)         

        return config

In [ ]:
show_doc(AutoRNN, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoRNN.default_config
config = dict(max_steps=2, val_check_steps=1, input_size=-1, encoder_hidden_size=8)
model = AutoRNN(h=12, config=config, num_samples=1, cpus=1)

model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoRNN(h=12, config=None, num_samples=1, cpus=1, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoRNN, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoRNN.get_default_config(h=12, backend='optuna')
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 1, 'val_check_steps': 1, 'input_size': -1, 'encoder_hidden_size': 8})
    return config

model = AutoRNN(h=12, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoRNN.get_default_config(h=12, backend='ray')
my_config['max_steps'] = 1
my_config['val_check_steps'] = 1
my_config['input_size'] = -1
my_config['encoder_hidden_size'] = 8
model = AutoRNN(h=12, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

In [ ]:
#| export
class AutoLSTM(BaseAuto):

    default_config = {
        "input_size_multiplier": [-1, 4, 16, 64],
        "inference_input_size_multiplier": [-1],
        "h": None,
        "encoder_hidden_size": tune.choice([50, 100, 200, 300]),
        "encoder_n_layers": tune.randint(1, 4),
        "context_size": tune.choice([5, 10, 50]),
        "decoder_hidden_size": tune.choice([64, 128, 256, 512]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "max_steps": tune.choice([500, 1000]),
        "batch_size": tune.choice([16, 32]),
        "loss": None,
        "random_seed": tune.randint(1, 20)
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None,
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None):

        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend)         

        super(AutoLSTM, self).__init__(
              cls_model=LSTM,
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples,
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,            
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series=None):
        config = cls.default_config.copy()
        config['input_size'] = tune.choice([h*x \
                        for x in config['input_size_multiplier']])
        config['inference_input_size'] = tune.choice([h*x \
                        for x in config['inference_input_size_multiplier']])
        del config['input_size_multiplier'], config['inference_input_size_multiplier']
        if backend == 'optuna':
            config = cls._ray_config_to_optuna(config)         

        return config

In [ ]:
show_doc(AutoLSTM, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoLSTM.default_config
config = dict(max_steps=2, val_check_steps=1, input_size=-1, encoder_hidden_size=8)
model = AutoLSTM(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoLSTM(h=12, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoLSTM, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoLSTM.get_default_config(h=12, backend='optuna')
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 1, 'val_check_steps': 1, 'input_size': -1, 'encoder_hidden_size': 8})
    return config

model = AutoLSTM(h=12, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoLSTM.get_default_config(h=12, backend='ray')
my_config['max_steps'] = 1
my_config['val_check_steps'] = 1
my_config['input_size'] = -1
my_config['encoder_hidden_size'] = 8
model = AutoLSTM(h=12, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

In [ ]:
#| export
class AutoGRU(BaseAuto):

    default_config = {
        "input_size_multiplier": [-1, 4, 16, 64],
        "inference_input_size_multiplier": [-1],
        "h": None,
        "encoder_hidden_size": tune.choice([50, 100, 200, 300]),
        "encoder_n_layers": tune.randint(1, 4),
        "context_size": tune.choice([5, 10, 50]),
        "decoder_hidden_size": tune.choice([64, 128, 256, 512]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "max_steps": tune.choice([500, 1000]),
        "batch_size": tune.choice([16, 32]),
        "loss": None,
        "random_seed": tune.randint(1, 20)
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None,
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend)         

        super(AutoGRU, self).__init__(
              cls_model=GRU,
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config, 
              search_alg=search_alg,
              num_samples=num_samples,
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,            
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series=None):
        config = cls.default_config.copy()
        config['input_size'] = tune.choice([h*x \
                        for x in config['input_size_multiplier']])
        config['inference_input_size'] = tune.choice([h*x \
                        for x in config['inference_input_size_multiplier']])
        del config['input_size_multiplier'], config['inference_input_size_multiplier']
        if backend == 'optuna':
            config = cls._ray_config_to_optuna(config)         

        return config        

In [ ]:
show_doc(AutoGRU, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoGRU.default_config
config = dict(max_steps=2, val_check_steps=1, input_size=-1, encoder_hidden_size=8)
model = AutoGRU(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoGRU(h=12, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoGRU, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoGRU.get_default_config(h=12, backend='optuna')
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 1, 'val_check_steps': 1, 'input_size': -1, 'encoder_hidden_size': 8})
    return config

model = AutoGRU(h=12, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoGRU.get_default_config(h=12, backend='ray')
my_config['max_steps'] = 1
my_config['val_check_steps'] = 1
my_config['input_size'] = -1
my_config['encoder_hidden_size'] = 8
model = AutoGRU(h=12, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

In [ ]:
#| export
class AutoTCN(BaseAuto):

    default_config = {
        "input_size_multiplier": [-1, 4, 16, 64],
        "inference_input_size_multiplier": [-1],
        "h": None,
        "encoder_hidden_size": tune.choice([50, 100, 200, 300]),
        "context_size": tune.choice([5, 10, 50]),
        "decoder_hidden_size": tune.choice([64, 128]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "max_steps": tune.choice([500, 1000]),
        "batch_size": tune.choice([16, 32]),
        "loss": None,
        "random_seed": tune.randint(1, 20)
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None,
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend)         

        super(AutoTCN, self).__init__(
              cls_model=TCN,
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples,
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,            
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series=None):
        config = cls.default_config.copy()
        config['input_size'] = tune.choice([h*x \
                        for x in config['input_size_multiplier']])
        config['inference_input_size'] = tune.choice([h*x \
                        for x in config['inference_input_size_multiplier']])
        del config['input_size_multiplier'], config['inference_input_size_multiplier']
        if backend == 'optuna':
            config = cls._ray_config_to_optuna(config)         

        return config        

In [ ]:
show_doc(AutoTCN, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoTCN.default_config
config = dict(max_steps=2, val_check_steps=1, input_size=-1, encoder_hidden_size=8)
model = AutoTCN(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoTCN(h=12, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoTCN, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoTCN.get_default_config(h=12, backend='optuna')
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 1, 'val_check_steps': 1, 'input_size': -1, 'encoder_hidden_size': 8})
    return config

model = AutoTCN(h=12, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoTCN.get_default_config(h=12, backend='ray')
my_config['max_steps'] = 1
my_config['val_check_steps'] = 1
my_config['input_size'] = -1
my_config['encoder_hidden_size'] = 8
model = AutoTCN(h=12, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

In [ ]:
#| export
class AutoDeepAR(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4, 5],
        "h": None,
        "lstm_hidden_size": tune.choice([32, 64, 128, 256]),
        "lstm_n_layers": tune.randint(1, 4),
        "lstm_dropout": tune.uniform(0.0, 0.5),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice(['robust', 'minmax1']),
        "max_steps": tune.choice([500, 1000, 2000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "windows_batch_size": tune.choice([128, 256, 512, 1024]),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 loss=DistributionLoss(distribution='StudentT', level=[80, 90], return_params=False),
                 valid_loss=MQLoss(level=[80, 90]),
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend)          

        super(AutoDeepAR, self).__init__(
              cls_model=DeepAR, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,            
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series=None):
        config = cls.default_config.copy()
        config['input_size'] = tune.choice([h*x \
                        for x in config['input_size_multiplier']])
        config['step_size'] = tune.choice([1, h])        
        del config['input_size_multiplier']
        if backend == 'optuna':
            config = cls._ray_config_to_optuna(config)         

        return config        

In [ ]:
show_doc(AutoDeepAR, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoNHITS.default_config
config = dict(max_steps=1, val_check_steps=1, input_size=12, lstm_hidden_size=8)
model = AutoDeepAR(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoDeepAR(h=12, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoDeepAR, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoDeepAR.get_default_config(h=12, backend='optuna')
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 1, 'val_check_steps': 1, 'input_size': 12, 'lstm_hidden_size': 8})
    return config

model = AutoDeepAR(h=12, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoDeepAR.get_default_config(h=12, backend='ray')
my_config['max_steps'] = 1
my_config['val_check_steps'] = 1
my_config['input_size'] = 12
my_config['lstm_hidden_size'] = 8
model = AutoDeepAR(h=12, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

In [ ]:
#| export
class AutoDilatedRNN(BaseAuto):

    default_config = {
        "input_size_multiplier": [-1, 4, 16, 64],
        "inference_input_size_multiplier": [-1],
        "h": None,
        "cell_type": tune.choice(['LSTM', 'GRU']),
        "encoder_hidden_size": tune.choice([50, 100, 200, 300]),
        "dilations": tune.choice([ [[1, 2], [4, 8]], [[1, 2, 4, 8]] ]),
        "context_size": tune.choice([5, 10, 50]),
        "decoder_hidden_size": tune.choice([64, 128, 256, 512]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "max_steps": tune.choice([500, 1000]),
        "batch_size": tune.choice([16, 32]),
        "loss": None,
        "random_seed": tune.randint(1, 20)
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None,
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend)         

        super(AutoDilatedRNN, self).__init__(
              cls_model=DilatedRNN,
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,
         )
        
    @classmethod
    def get_default_config(cls, h, backend, n_series=None):
        config = cls.default_config.copy()
        config['input_size'] = tune.choice([h*x \
                        for x in config['input_size_multiplier']])
        config['inference_input_size'] = tune.choice([h*x \
                        for x in config['inference_input_size_multiplier']])
        del config['input_size_multiplier'], config['inference_input_size_multiplier']
        if backend == 'optuna':
            config = cls._ray_config_to_optuna(config)         

        return config        

In [ ]:
show_doc(AutoDilatedRNN, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoDilatedRNN.default_config
config = dict(max_steps=2, val_check_steps=1, input_size=-1, encoder_hidden_size=8)
model = AutoDilatedRNN(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoDilatedRNN(h=12, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoDilatedRNN, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoDilatedRNN.get_default_config(h=12, backend='optuna')
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 1, 'val_check_steps': 1, 'input_size': -1, 'encoder_hidden_size': 8})
    return config

model = AutoDilatedRNN(h=12, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoDilatedRNN.get_default_config(h=12, backend='ray')
my_config['max_steps'] = 1
my_config['val_check_steps'] = 1
my_config['input_size'] = -1
my_config['encoder_hidden_size'] = 8
model = AutoDilatedRNN(h=12, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

In [ ]:
#| export
class AutoBiTCN(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4, 5],
        "h": None,
        "hidden_size": tune.choice([16, 32]),
        "dropout": tune.uniform(0.0, 0.99),  
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice([None, 'robust', 'standard']),
        "max_steps": tune.choice([500, 1000, 2000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "windows_batch_size": tune.choice([128, 256, 512, 1024]),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend)          

        super(AutoBiTCN, self).__init__(
              cls_model=BiTCN, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,            
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series=None):
        config = cls.default_config.copy()
        config['input_size'] = tune.choice([h*x \
                        for x in config['input_size_multiplier']])
        config['step_size'] = tune.choice([1, h])        
        del config['input_size_multiplier']
        if backend == 'optuna':
            config = cls._ray_config_to_optuna(config)         

        return config   

In [ ]:
show_doc(AutoBiTCN, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoNHITS.default_config
config = dict(max_steps=1, val_check_steps=1, input_size=12, hidden_size=8)
model = AutoBiTCN(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoBiTCN(h=12, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoBiTCN, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoBiTCN.get_default_config(h=12, backend='optuna')
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 1, 'val_check_steps': 1, 'input_size': 12, 'hidden_size': 8})
    return config

model = AutoBiTCN(h=12, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoBiTCN.get_default_config(h=12, backend='ray')
my_config['max_steps'] = 1
my_config['val_check_steps'] = 1
my_config['input_size'] = 12
my_config['hidden_size'] = 8
model = AutoBiTCN(h=12, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

## B. MLP-Based

In [ ]:
#| export
class AutoMLP(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4, 5],
        "h": None,
        "hidden_size": tune.choice( [256, 512, 1024] ),
        "num_layers": tune.randint(2, 6),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice([None, 'robust', 'standard']),
        "max_steps": tune.choice([500, 1000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "windows_batch_size": tune.choice([128, 256, 512, 1024]),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,     
                 config=None,
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None):

        # Define search space, input/output sizes       
        if config is None:
            config = self.get_default_config(h=h, backend=backend)          

        super(AutoMLP, self).__init__(
              cls_model=MLP,
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config, 
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,            
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series=None):
        config = cls.default_config.copy()
        config['input_size'] = tune.choice([h*x \
                        for x in config['input_size_multiplier']])
        config['step_size'] = tune.choice([1, h])        
        del config['input_size_multiplier']
        if backend == 'optuna':
            config = cls._ray_config_to_optuna(config)         

        return config 

In [ ]:
show_doc(AutoMLP, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoMLP.default_config
config = dict(max_steps=2, val_check_steps=1, input_size=12, hidden_size=8)
model = AutoMLP(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoMLP(h=12, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoMLP, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoMLP.get_default_config(h=12, backend='optuna')
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 2, 'val_check_steps': 1, 'input_size': 12, 'hidden_size': 8})
    return config

model = AutoMLP(h=12, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoMLP.get_default_config(h=12, backend='ray')
my_config['max_steps'] = 2
my_config['val_check_steps'] = 1
my_config['input_size'] = 12
my_config['hidden_size'] = 8
model = AutoMLP(h=12, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

In [ ]:
#| export
class AutoNBEATS(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4, 5],
        "h": None,
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice([None, 'robust', 'standard']),
        "max_steps": tune.choice([500, 1000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "windows_batch_size": tune.choice([128, 256, 512, 1024]),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None):
        
        # Define search space, input/output sizes 
        if config is None:
            config = self.get_default_config(h=h, backend=backend)          

        super(AutoNBEATS, self).__init__(
              cls_model=NBEATS, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,            
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series=None):
        config = cls.default_config.copy()
        config['input_size'] = tune.choice([h*x \
                        for x in config['input_size_multiplier']])
        config['step_size'] = tune.choice([1, h])        
        del config['input_size_multiplier']
        if backend == 'optuna':
            config = cls._ray_config_to_optuna(config)         

        return config                

In [ ]:
show_doc(AutoNBEATS, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoNBEATS.default_config
config = dict(max_steps=2, val_check_steps=1, input_size=12,
              mlp_units=3*[[8, 8]])
model = AutoNBEATS(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoNBEATS(h=12, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoNBEATS, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoNBEATS.get_default_config(h=12, backend='optuna')
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 2, 'val_check_steps': 1, 'input_size': 12, 'mlp_units': 3 * [[8, 8]]})
    return config

model = AutoNBEATS(h=12, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoNBEATS.get_default_config(h=12, backend='ray')
my_config['max_steps'] = 2
my_config['val_check_steps'] = 1
my_config['input_size'] = 12
my_config['mlp_units'] = 3 * [[8, 8]]
model = AutoNBEATS(h=12, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

In [ ]:
#| export
class AutoNBEATSx(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4, 5],
        "h": None,
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice([None, 'robust', 'standard']),
        "max_steps": tune.choice([500, 1000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "windows_batch_size": tune.choice([128, 256, 512, 1024]),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend)          

        super(AutoNBEATSx, self).__init__(
              cls_model=NBEATSx,
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,            
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series=None):
        config = cls.default_config.copy()
        config['input_size'] = tune.choice([h*x \
                        for x in config['input_size_multiplier']])
        config['step_size'] = tune.choice([1, h])        
        del config['input_size_multiplier']
        if backend == 'optuna':
            config = cls._ray_config_to_optuna(config)         

        return config           

In [ ]:
show_doc(AutoNBEATSx, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoNBEATS.default_config
config = dict(max_steps=2, val_check_steps=1, input_size=12,
              mlp_units=3*[[8, 8]])
model = AutoNBEATSx(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoNBEATSx(h=12, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoNBEATSx, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoNBEATSx.get_default_config(h=12, backend='optuna')
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 2, 'val_check_steps': 1, 'input_size': 12, 'mlp_units': 3 * [[8, 8]]})
    return config

model = AutoNBEATSx(h=12, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoNBEATSx.get_default_config(h=12, backend='ray')
my_config['max_steps'] = 2
my_config['val_check_steps'] = 1
my_config['input_size'] = 12
my_config['mlp_units'] = 3 * [[8, 8]]
model = AutoNBEATSx(h=12, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

In [ ]:
#| export
class AutoNHITS(BaseAuto):

    default_config = {
       "input_size_multiplier": [1, 2, 3, 4, 5],
       "h": None,
       "n_pool_kernel_size": tune.choice([[2, 2, 1], 3*[1], 3*[2], 3*[4], 
                                         [8, 4, 1], [16, 8, 1]]),
       "n_freq_downsample": tune.choice([[168, 24, 1], [24, 12, 1], 
                                         [180, 60, 1], [60, 8, 1], 
                                         [40, 20, 1], [1, 1, 1]]),
       "learning_rate": tune.loguniform(1e-4, 1e-1),
       "scaler_type": tune.choice([None, 'robust', 'standard']),
       "max_steps": tune.quniform(lower=500, upper=1500, q=100),
       "batch_size": tune.choice([32, 64, 128, 256]),
       "windows_batch_size": tune.choice([128, 256, 512, 1024]),
       "loss": None,
       "random_seed": tune.randint(lower=1, upper=20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None,
                ):

        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend)                     

        super(AutoNHITS, self).__init__(
              cls_model=NHITS, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples,
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series=None):
        config = cls.default_config.copy()
        config['input_size'] = tune.choice([h*x \
                        for x in config['input_size_multiplier']])
        config['step_size'] = tune.choice([1, h])        
        del config['input_size_multiplier']
        if backend == 'optuna':
            config = cls._ray_config_to_optuna(config)         

        return config        

In [ ]:
show_doc(AutoNHITS, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoNHITS.default_config
config = dict(max_steps=2, val_check_steps=1, input_size=12, 
              mlp_units=3 * [[8, 8]])
model = AutoNHITS(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoNHITS(h=12, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoNHITS, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoNHITS.get_default_config(h=12, backend='optuna')
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 2, 'val_check_steps': 1, 'input_size': 12, 'mlp_units': 3 * [[8, 8]]})
    return config

model = AutoNHITS(h=12, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoNHITS.get_default_config(h=12, backend='ray')
my_config['max_steps'] = 2
my_config['val_check_steps'] = 1
my_config['input_size'] = 12
my_config['mlp_units'] = 3 * [[8, 8]]
model = AutoNHITS(h=12, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

In [ ]:
#| export
class AutoDLinear(BaseAuto):

    default_config = {
       "input_size_multiplier": [1, 2, 3, 4, 5],
       "h": None,
       "moving_avg_window": tune.choice([11, 25, 51]),
       "learning_rate": tune.loguniform(1e-4, 1e-1),
       "scaler_type": tune.choice([None, 'robust', 'standard']),
       "max_steps": tune.quniform(lower=500, upper=1500, q=100),
       "batch_size": tune.choice([32, 64, 128, 256]),
       "windows_batch_size": tune.choice([128, 256, 512, 1024]),
       "loss": None,
       "random_seed": tune.randint(lower=1, upper=20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None,
                ):

        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend)                   

        super(AutoDLinear, self).__init__(
              cls_model=DLinear, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples,
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series=None):
        config = cls.default_config.copy()
        config['input_size'] = tune.choice([h*x \
                        for x in config['input_size_multiplier']])
        config['step_size'] = tune.choice([1, h])        
        del config['input_size_multiplier']
        if backend == 'optuna':
            config = cls._ray_config_to_optuna(config)         

        return config           

In [ ]:
show_doc(AutoDLinear, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoDLinear.default_config
config = dict(max_steps=2, val_check_steps=1, input_size=12)
model = AutoDLinear(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoDLinear(h=12, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoDLinear, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoDLinear.get_default_config(h=12, backend='optuna')
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 2, 'val_check_steps': 1, 'input_size': 12})
    return config

model = AutoDLinear(h=12, config=my_config_new, backend='optuna', cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoDLinear.get_default_config(h=12, backend='ray')
my_config['max_steps'] = 2
my_config['val_check_steps'] = 1
my_config['input_size'] = 12
model = AutoDLinear(h=12, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

In [ ]:
#| export
class AutoNLinear(BaseAuto):

    default_config = {
       "input_size_multiplier": [1, 2, 3, 4, 5],
       "h": None,
       "learning_rate": tune.loguniform(1e-4, 1e-1),
       "scaler_type": tune.choice([None, 'robust', 'standard']),
       "max_steps": tune.quniform(lower=500, upper=1500, q=100),
       "batch_size": tune.choice([32, 64, 128, 256]),
       "windows_batch_size": tune.choice([128, 256, 512, 1024]),
       "loss": None,
       "random_seed": tune.randint(lower=1, upper=20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None,
                ):

        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend)                 

        super(AutoNLinear, self).__init__(
              cls_model=NLinear, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples,
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series=None):
        config = cls.default_config.copy()
        config['input_size'] = tune.choice([h*x \
                        for x in config['input_size_multiplier']])
        config['step_size'] = tune.choice([1, h])        
        del config['input_size_multiplier']
        if backend == 'optuna':
            config = cls._ray_config_to_optuna(config)         

        return config           

In [ ]:
show_doc(AutoNLinear, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoNLinear.default_config
config = dict(max_steps=2, val_check_steps=1, input_size=12)
model = AutoNLinear(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoNLinear(h=12, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoNLinear, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoNLinear.get_default_config(h=12, backend='optuna')
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 2, 'val_check_steps': 1, 'input_size': 12})
    return config

model = AutoNLinear(h=12, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoNLinear.get_default_config(h=12, backend='ray')
my_config['max_steps'] = 2
my_config['val_check_steps'] = 1
my_config['input_size'] = 12
model = AutoNLinear(h=12, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

In [ ]:
#| export
class AutoTiDE(BaseAuto):

    default_config = {
       "input_size_multiplier": [1, 2, 3, 4, 5],
       "h": None,
       "hidden_size": tune.choice([256, 512, 1024]),
       "decoder_output_dim": tune.choice([8, 16, 32]),
       "temporal_decoder_dim": tune.choice([32, 64, 128]),
       "num_encoder_layers": tune.choice([1, 2, 3]),
       "num_decoder_layers": tune.choice([1, 2, 3]),
       "temporal_width": tune.choice([4, 8, 16]),
       "dropout":tune.choice([0.0, 0.1, 0.2, 0.3, 0.5]),
       "layernorm": tune.choice([True, False]),
       "learning_rate": tune.loguniform(1e-5, 1e-2),
       "scaler_type": tune.choice([None, 'robust', 'standard']),
       "max_steps": tune.quniform(lower=500, upper=1500, q=100),
       "batch_size": tune.choice([32, 64, 128, 256]),
       "windows_batch_size": tune.choice([128, 256, 512, 1024]),
       "loss": None,
       "random_seed": tune.randint(lower=1, upper=20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None,
                ):

        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend)                 

        super(AutoTiDE, self).__init__(
              cls_model=TiDE, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples,
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series=None):
        config = cls.default_config.copy()
        config['input_size'] = tune.choice([h*x \
                        for x in config['input_size_multiplier']])
        config['step_size'] = tune.choice([1, h])        
        del config['input_size_multiplier']
        if backend == 'optuna':
            config = cls._ray_config_to_optuna(config)         

        return config    

In [ ]:
show_doc(AutoTiDE, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoTiDE.default_config
config = dict(max_steps=2, val_check_steps=1, input_size=12)
model = AutoTiDE(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoTiDE(h=12, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoTiDE, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoTiDE.get_default_config(h=12, backend='optuna')
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 2, 'val_check_steps': 1, 'input_size': 12})
    return config

model = AutoTiDE(h=12, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoTiDE.get_default_config(h=12, backend='ray')
my_config['max_steps'] = 2
my_config['val_check_steps'] = 1
my_config['input_size'] = 12
model = AutoTiDE(h=12, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

In [ ]:
#| export
class AutoDeepNPTS(BaseAuto):

    default_config = {
       "input_size_multiplier": [1, 2, 3, 4, 5],
       "h": None,
       "hidden_size": tune.choice([16, 32, 64]),
       "dropout": tune.uniform(0.0, 0.99),
       "n_layers": tune.choice([1, 2, 4]),
       "learning_rate": tune.loguniform(1e-4, 1e-1),
       "scaler_type": tune.choice([None, 'robust', 'standard']),
       "max_steps": tune.quniform(lower=500, upper=1500, q=100),
       "batch_size": tune.choice([32, 64, 128, 256]),
       "windows_batch_size": tune.choice([128, 256, 512, 1024]),
       "loss": None,
       "random_seed": tune.randint(lower=1, upper=20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None,
                ):

        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend)                 

        super(AutoDeepNPTS, self).__init__(
              cls_model=DeepNPTS, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples,
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series=None):
        config = cls.default_config.copy()
        config['input_size'] = tune.choice([h*x \
                        for x in config['input_size_multiplier']])
        config['step_size'] = tune.choice([1, h])        
        del config['input_size_multiplier']
        if backend == 'optuna':
            config = cls._ray_config_to_optuna(config)         

        return config  

In [ ]:
show_doc(AutoDeepNPTS, title_level=3)

NameError: name 'AutoDeepNPTS' is not defined

In [ ]:
%%capture
# Use your own config or AutoDeepNPTS.default_config
config = dict(max_steps=2, val_check_steps=1, input_size=12)
model = AutoDeepNPTS(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoDeepNPTS(h=12, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoDeepNPTS, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoDeepNPTS.get_default_config(h=12, backend='optuna')
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 2, 'val_check_steps': 1, 'input_size': 12})
    return config

model = AutoDeepNPTS(h=12, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoDeepNPTS.get_default_config(h=12, backend='ray')
my_config['max_steps'] = 2
my_config['val_check_steps'] = 1
my_config['input_size'] = 12
model = AutoDeepNPTS(h=12, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

In [ ]:
#| export
class AutoKAN(BaseAuto):

    default_config = {
       "input_size_multiplier": [1, 2, 3, 4, 5],
       "h": None,
       "hidden_size": tune.choice([64, 128, 256, 512]),
       "learning_rate": tune.loguniform(1e-4, 1e-1),
       "scaler_type": tune.choice([None, 'robust', 'standard']),
       "max_steps": tune.quniform(lower=500, upper=1500, q=100),
       "batch_size": tune.choice([32, 64, 128, 256]),
       "windows_batch_size": tune.choice([128, 256, 512, 1024]),
       "loss": None,
       "random_seed": tune.randint(lower=1, upper=20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None,
                ):

        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend)                 

        super(AutoKAN, self).__init__(
              cls_model=KAN, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples,
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series=None):
        config = cls.default_config.copy()
        config['input_size'] = tune.choice([h*x \
                        for x in config['input_size_multiplier']])
        config['step_size'] = tune.choice([1, h])        
        del config['input_size_multiplier']
        if backend == 'optuna':
            config = cls._ray_config_to_optuna(config)         

        return config  

In [ ]:
show_doc(AutoKAN, title_level=3)

---

### AutoKAN

>      AutoKAN (h, loss=MAE(), valid_loss=None, config=None,
>               search_alg=<ray.tune.search.basic_variant.BasicVariantGenerator
>               object at 0x176b51180>, num_samples=10, refit_with_val=False,
>               cpus=11, gpus=0, verbose=False, alias=None, backend='ray',
>               callbacks=None)

Class for Automatic Hyperparameter Optimization, it builds on top of `ray` to
give access to a wide variety of hyperparameter optimization tools ranging
from classic grid search, to Bayesian optimization and HyperBand algorithm.

The validation loss to be optimized is defined by the `config['loss']` dictionary
value, the config also contains the rest of the hyperparameter search space.

It is important to note that the success of this hyperparameter optimization
heavily relies on a strong correlation between the validation and test periods.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| h | int |  | Forecast horizon |
| loss | MAE | MAE() | Instantiated train loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html). |
| valid_loss | NoneType | None | Instantiated valid loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html). |
| config | NoneType | None | Dictionary with ray.tune defined search space or function that takes an optuna trial and returns a configuration dict. |
| search_alg | BasicVariantGenerator | <ray.tune.search.basic_variant.BasicVariantGenerator object> | For ray see https://docs.ray.io/en/latest/tune/api_docs/suggestion.html<br>For optuna see https://optuna.readthedocs.io/en/stable/reference/samplers/index.html. |
| num_samples | int | 10 | Number of hyperparameter optimization steps/samples. |
| refit_with_val | bool | False | Refit of best model should preserve val_size. |
| cpus | int | 11 | Number of cpus to use during optimization. Only used with ray tune. |
| gpus | int | 0 | Number of gpus to use during optimization, default all available. Only used with ray tune. |
| verbose | bool | False | Track progress. |
| alias | NoneType | None | Custom name of the model. |
| backend | str | ray | Backend to use for searching the hyperparameter space, can be either 'ray' or 'optuna'. |
| callbacks | NoneType | None | List of functions to call during the optimization process.<br>ray reference: https://docs.ray.io/en/latest/tune/tutorials/tune-metrics.html<br>optuna reference: https://optuna.readthedocs.io/en/stable/tutorial/20_recipes/007_optuna_callback.html |

In [ ]:
%%capture
# Use your own config or AutoDeepNPTS.default_config
config = dict(max_steps=2, val_check_steps=1, input_size=12)
model = AutoKAN(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoKAN(h=12, config=None, backend='optuna')

NameError: name 'dataset' is not defined

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoKAN, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoKAN.get_default_config(h=12, backend='optuna')
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 2, 'val_check_steps': 1, 'input_size': 12})
    return config

model = AutoKAN(h=12, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoKAN.get_default_config(h=12, backend='ray')
my_config['max_steps'] = 2
my_config['val_check_steps'] = 1
my_config['input_size'] = 12
model = AutoKAN(h=12, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

## C. Transformer-Based

In [ ]:
#| export
class AutoTFT(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4, 5],
        "h": None,
        "hidden_size": tune.choice([64, 128, 256]),
        "n_head": tune.choice([4, 8]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice([None, 'robust', 'standard']),
        "max_steps": tune.choice([500, 1000, 2000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "windows_batch_size": tune.choice([128, 256, 512, 1024]),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend)             

        super(AutoTFT, self).__init__(
              cls_model=TFT, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,            
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series=None):
        config = cls.default_config.copy()
        config['input_size'] = tune.choice([h*x \
                        for x in config['input_size_multiplier']])
        config['step_size'] = tune.choice([1, h])        
        del config['input_size_multiplier']
        if backend == 'optuna':
            config = cls._ray_config_to_optuna(config)         

        return config

In [ ]:
show_doc(AutoTFT, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoNHITS.default_config
config = dict(max_steps=1, val_check_steps=1, input_size=12, hidden_size=8)
model = AutoTFT(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoTFT(h=12, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoTFT, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoTFT.get_default_config(h=12, backend='optuna')
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 1, 'val_check_steps': 1, 'input_size': 12, 'hidden_size': 8})
    return config

model = AutoTFT(h=12, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoTFT.get_default_config(h=12, backend='ray')
my_config['max_steps'] = 1
my_config['val_check_steps'] = 1
my_config['input_size'] = 12
my_config['hidden_size'] = 8
model = AutoTFT(h=12, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

In [ ]:
#| export
class AutoVanillaTransformer(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4, 5],
        "h": None,
        "hidden_size": tune.choice([64, 128, 256]),
        "n_head": tune.choice([4, 8]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice([None, 'robust', 'standard']),
        "max_steps": tune.choice([500, 1000, 2000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "windows_batch_size": tune.choice([128, 256, 512, 1024]),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend)          

        super(AutoVanillaTransformer, self).__init__(
              cls_model=VanillaTransformer, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,            
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series=None):
        config = cls.default_config.copy()
        config['input_size'] = tune.choice([h*x \
                        for x in config['input_size_multiplier']])
        config['step_size'] = tune.choice([1, h])        
        del config['input_size_multiplier']
        if backend == 'optuna':
            config = cls._ray_config_to_optuna(config)         

        return config   

In [ ]:
show_doc(AutoVanillaTransformer, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoNHITS.default_config
config = dict(max_steps=1, val_check_steps=1, input_size=12, hidden_size=8)
model = AutoVanillaTransformer(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoVanillaTransformer(h=12, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoVanillaTransformer, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoVanillaTransformer.get_default_config(h=12, backend='optuna')
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 1, 'val_check_steps': 1, 'input_size': 12, 'hidden_size': 8})
    return config

model = AutoVanillaTransformer(h=12, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoVanillaTransformer.get_default_config(h=12, backend='ray')
my_config['max_steps'] = 1
my_config['val_check_steps'] = 1
my_config['input_size'] = 12
my_config['hidden_size'] = 8
model = AutoVanillaTransformer(h=12, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

In [ ]:
#| export
class AutoInformer(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4, 5],
        "h": None,
        "hidden_size": tune.choice([64, 128, 256]),
        "n_head": tune.choice([4, 8]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice([None, 'robust', 'standard']),
        "max_steps": tune.choice([500, 1000, 2000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "windows_batch_size": tune.choice([128, 256, 512, 1024]),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend)          

        super(AutoInformer, self).__init__(
              cls_model=Informer, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,            
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series=None):
        config = cls.default_config.copy()
        config['input_size'] = tune.choice([h*x \
                        for x in config['input_size_multiplier']])
        config['step_size'] = tune.choice([1, h])        
        del config['input_size_multiplier']
        if backend == 'optuna':
            config = cls._ray_config_to_optuna(config)         

        return config                

In [ ]:
show_doc(AutoInformer, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoNHITS.default_config
config = dict(max_steps=1, val_check_steps=1, input_size=12, hidden_size=8)
model = AutoInformer(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoInformer(h=12, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoInformer, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoInformer.get_default_config(h=12, backend='optuna')
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 1, 'val_check_steps': 1, 'input_size': 12, 'hidden_size': 8})
    return config

model = AutoInformer(h=12, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoInformer.get_default_config(h=12, backend='ray')
my_config['max_steps'] = 1
my_config['val_check_steps'] = 1
my_config['input_size'] = 12
my_config['hidden_size'] = 8
model = AutoInformer(h=12, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

In [ ]:
#| export
class AutoAutoformer(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4, 5],
        "h": None,
        "hidden_size": tune.choice([64, 128, 256]),
        "n_head": tune.choice([4, 8]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice([None, 'robust', 'standard']),
        "max_steps": tune.choice([500, 1000, 2000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "windows_batch_size": tune.choice([128, 256, 512, 1024]),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend)          

        super(AutoAutoformer, self).__init__(
              cls_model=Autoformer, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,            
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series=None):
        config = cls.default_config.copy()
        config['input_size'] = tune.choice([h*x \
                        for x in config['input_size_multiplier']])
        config['step_size'] = tune.choice([1, h])        
        del config['input_size_multiplier']
        if backend == 'optuna':
            config = cls._ray_config_to_optuna(config)         

        return config                

In [ ]:
show_doc(AutoAutoformer, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoNHITS.default_config
config = dict(max_steps=1, val_check_steps=1, input_size=12, hidden_size=8)
model = AutoAutoformer(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoAutoformer(h=12, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoAutoformer, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoAutoformer.get_default_config(h=12, backend='optuna')
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 1, 'val_check_steps': 1, 'input_size': 12, 'hidden_size': 8})
    return config

model = AutoAutoformer(h=12, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoAutoformer.get_default_config(h=12, backend='ray')
my_config['max_steps'] = 1
my_config['val_check_steps'] = 1
my_config['input_size'] = 12
my_config['hidden_size'] = 8
model = AutoAutoformer(h=12, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

In [ ]:
#| export
class AutoFEDformer(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4, 5],
        "h": None,
        "hidden_size": tune.choice([64, 128, 256]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice([None, 'robust', 'standard']),
        "max_steps": tune.choice([500, 1000, 2000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "windows_batch_size": tune.choice([128, 256, 512, 1024]),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None):
        
        # Define search space, input/output sizes    
        if config is None:
            config = self.get_default_config(h=h, backend=backend)          

        super(AutoFEDformer, self).__init__(
              cls_model=FEDformer, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,            
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series=None):
        config = cls.default_config.copy()
        config['input_size'] = tune.choice([h*x \
                        for x in config['input_size_multiplier']])
        config['step_size'] = tune.choice([1, h])        
        del config['input_size_multiplier']
        if backend == 'optuna':
            config = cls._ray_config_to_optuna(config)         

        return config   

In [ ]:
show_doc(AutoFEDformer, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoNHITS.default_config
config = dict(max_steps=1, val_check_steps=1, input_size=12, hidden_size=64)
model = AutoFEDformer(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoFEDformer(h=12, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoFEDformer, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoFEDformer.get_default_config(h=12, backend='optuna')
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 1, 'val_check_steps': 1, 'input_size': 12, 'hidden_size': 64})
    return config

model = AutoFEDformer(h=12, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoFEDformer.get_default_config(h=12, backend='ray')
my_config['max_steps'] = 1
my_config['val_check_steps'] = 1
my_config['input_size'] = 12
my_config['hidden_size'] = 64
model = AutoFEDformer(h=12, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

In [ ]:
#| export
class AutoPatchTST(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3],
        "h": None,
        "hidden_size": tune.choice([16, 128, 256]),
        "n_heads": tune.choice([4, 16]),
        "patch_len": tune.choice([16, 24]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice([None, 'robust', 'standard']),
        "revin": tune.choice([False, True]),
        "max_steps": tune.choice([500, 1000, 5000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "windows_batch_size": tune.choice([128, 256, 512, 1024]),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend)           

        super(AutoPatchTST, self).__init__(
              cls_model=PatchTST, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,            
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series=None):
        config = cls.default_config.copy()
        config['input_size'] = tune.choice([h * x \
                        for x in config['input_size_multiplier']])  
        config['step_size'] = tune.choice([1, h])        
        del config['input_size_multiplier']
        if backend == 'optuna':
            config = cls._ray_config_to_optuna(config)         

        return config         

In [ ]:
show_doc(AutoPatchTST, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoNHITS.default_config
config = dict(max_steps=1, val_check_steps=1, input_size=12, hidden_size=16)
model = AutoPatchTST(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoPatchTST(h=12, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoPatchTST, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoPatchTST.get_default_config(h=12, backend='optuna')
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 1, 'val_check_steps': 1, 'input_size': 12, 'hidden_size': 16})
    return config

model = AutoPatchTST(h=12, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoPatchTST.get_default_config(h=12, backend='ray')
my_config['max_steps'] = 1
my_config['val_check_steps'] = 1
my_config['input_size'] = 12
my_config['hidden_size'] = 16
model = AutoPatchTST(h=12, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

In [ ]:
#| export
class AutoiTransformer(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4, 5],
        "h": None,
        "n_series": None,
        "hidden_size": tune.choice([64, 128, 256]),
        "n_heads": tune.choice([4, 8]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice([None, 'robust', 'standard']),
        "max_steps": tune.choice([500, 1000, 2000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 n_series,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend, n_series=n_series)                 

        # Always use n_series from parameters, raise exception with Optuna because we can't enforce it
        if backend == 'ray':
            config['n_series'] = n_series
        elif backend == 'optuna':
            mock_trial = MockTrial()
            if ('n_series' in config(mock_trial) and config(mock_trial)['n_series'] != n_series) or ('n_series' not in config(mock_trial)):
                raise Exception(f"config needs 'n_series': {n_series}")           

        super(AutoiTransformer, self).__init__(
              cls_model=iTransformer, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,            
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series):
        config = cls.default_config.copy()        
        config['input_size'] = tune.choice([h * x \
                        for x in config["input_size_multiplier"]])

        # Rolling windows with step_size=1 or step_size=h
        # See `BaseWindows` and `BaseRNN`'s create_windows
        config['step_size'] = tune.choice([1, h])
        del config["input_size_multiplier"]
        if backend == 'optuna':
            # Always use n_series from parameters
            config['n_series'] = n_series
            config = cls._ray_config_to_optuna(config)           

        return config         

In [ ]:
show_doc(AutoiTransformer, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoiTransformer.default_config
config = dict(max_steps=1, val_check_steps=1, input_size=12, hidden_size=16)
model = AutoiTransformer(h=12, n_series=1, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoiTransformer(h=12, n_series=1, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoiTransformer, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoiTransformer.get_default_config(h=12, n_series=1, backend='optuna')
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 1, 'val_check_steps': 1, 'input_size': 12, 'hidden_size': 16})
    return config

model = AutoiTransformer(h=12, n_series=1, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoiTransformer.get_default_config(h=12, n_series=1, backend='ray')
my_config['max_steps'] = 1
my_config['val_check_steps'] = 1
my_config['input_size'] = 12
my_config['hidden_size'] = 16
model = AutoiTransformer(h=12, n_series=1, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

## D. CNN Based

In [ ]:
#| export
class AutoTimesNet(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4, 5],
        "h": None,
        "hidden_size": tune.choice([32, 64, 128]),
        "conv_hidden_size": tune.choice([32, 64, 128]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice(['robust', 'standard']),
        "max_steps": tune.choice([500, 1000, 2000]),
        "batch_size": tune.choice([32, 64, 128]),
        "windows_batch_size": tune.choice([32, 64, 128, 256]),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend)                  

        super(AutoTimesNet, self).__init__(
              cls_model=TimesNet, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,            
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series=None):
        config = cls.default_config.copy()
        config['input_size'] = tune.choice([h*x \
                        for x in config['input_size_multiplier']])
        config['step_size'] = tune.choice([1, h])        
        del config['input_size_multiplier']
        if backend == 'optuna':
            config = cls._ray_config_to_optuna(config)         

        return config         

In [ ]:
show_doc(AutoTimesNet, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoTimesNet.default_config
config = dict(max_steps=1, val_check_steps=1, input_size=12, hidden_size=32)
model = AutoTimesNet(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoTimesNet(h=12, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoTimesNet, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoTimesNet.get_default_config(h=12, backend='optuna')
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 2, 'val_check_steps': 1, 'input_size': 12, 'hidden_size': 32})
    return config

model = AutoTimesNet(h=12, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoTimesNet.get_default_config(h=12, backend='ray')
my_config['max_steps'] = 2
my_config['val_check_steps'] = 1
my_config['input_size'] = 12
my_config['hidden_size'] = 32
model = AutoTimesNet(h=12, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

## E. Multivariate

In [ ]:
#| export
class AutoStemGNN(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4],
        "h": None,
        "n_series": None,
        "n_stacks": tune.choice([2]),
        "multi_layer": tune.choice([3, 5, 7]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice([None, 'robust', 'standard']),
        "max_steps": tune.choice([500, 1000, 2000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 n_series,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend, n_series=n_series)                  

        # Always use n_series from parameters, raise exception with Optuna because we can't enforce it
        if backend == 'ray':
            config['n_series'] = n_series
        elif backend == 'optuna':
            mock_trial = MockTrial()
            if ('n_series' in config(mock_trial) and config(mock_trial)['n_series'] != n_series) or ('n_series' not in config(mock_trial)):
                raise Exception(f"config needs 'n_series': {n_series}")

        super(AutoStemGNN, self).__init__(
              cls_model=StemGNN, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,            
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series):
        config = cls.default_config.copy()        
        config['input_size'] = tune.choice([h * x \
                        for x in config["input_size_multiplier"]])

        # Rolling windows with step_size=1 or step_size=h
        # See `BaseWindows` and `BaseRNN`'s create_windows
        config['step_size'] = tune.choice([1, h])
        del config["input_size_multiplier"]
        if backend == 'optuna':
            # Always use n_series from parameters
            config['n_series'] = n_series
            config = cls._ray_config_to_optuna(config)           

        return config        

In [ ]:
show_doc(AutoStemGNN, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoStemGNN.default_config
config = dict(max_steps=1, val_check_steps=1, input_size=12)
model = AutoStemGNN(h=12, n_series=1, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoStemGNN(h=12, n_series=1, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12
assert model.config(MockTrial())['n_series'] == 1

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoStemGNN, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoStemGNN.get_default_config(h=12, backend='optuna', n_series=1)
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 1, 'val_check_steps': 1, 'input_size': 12})
    return config

model = AutoStemGNN(h=12, n_series=1, config=my_config_new, backend='optuna')
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoStemGNN.get_default_config(h=12, backend='ray', n_series=1)
my_config['max_steps'] = 1
my_config['val_check_steps'] = 1
my_config['input_size'] = 12
model = AutoStemGNN(h=12, n_series=1, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

In [ ]:
#| export
class AutoHINT(BaseAuto):

    def __init__(self,
                 cls_model,
                 h,
                 loss,
                 valid_loss,
                 S,
                 config,
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 refit_with_val=False,
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None,
                 ):
        
        super(AutoHINT, self).__init__(
              cls_model=cls_model, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,
        )
        if backend == 'optuna':
            raise Exception("Optuna is not supported for AutoHINT.")

        # Validate presence of reconciliation strategy
        # parameter in configuration space
        if not ('reconciliation' in config.keys()):
            raise Exception("config needs reconciliation, \
                            try tune.choice(['BottomUp', 'MinTraceOLS', 'MinTraceWLS'])")
        self.S = S

    def _fit_model(self, cls_model, config,
                   dataset, val_size, test_size, distributed_config=None):
        # Overwrite _fit_model for HINT two-stage instantiation
        reconciliation = config.pop('reconciliation')
        base_model = cls_model(**config)
        model = HINT(h=base_model.h, model=base_model, 
                     S=self.S, reconciliation=reconciliation)
        model.test_size = test_size
        model = model.fit(
            dataset,
            val_size=val_size, 
            test_size=test_size,
            distributed_config=distributed_config,
        )
        return model
    
    @classmethod
    def get_default_config(cls, h, backend, n_series=None):
        raise Exception("AutoHINT has no default configuration.")


In [ ]:
show_doc(AutoHINT, title_level=3)

In [ ]:
#| hide
def sort_df_hier(Y_df, S_df):
    # NeuralForecast core, sorts unique_id lexicographically
    # by default, this class matches S_df and Y_hat_df order.    
    Y_df.unique_id = Y_df.unique_id.astype('category')
    Y_df.unique_id = Y_df.unique_id.cat.set_categories(S_df.index)
    Y_df = Y_df.sort_values(by=['unique_id', 'ds'])
    return Y_df

# -----Create synthetic dataset-----
np.random.seed(123)
train_steps = 20
num_levels = 7
level = np.arange(0, 100, 0.1)
qs = [[50-lv/2, 50+lv/2] for lv in level]
quantiles = np.sort(np.concatenate(qs)/100)

levels = ['Top', 'Mid1', 'Mid2', 'Bottom1', 'Bottom2', 'Bottom3', 'Bottom4']
unique_ids = np.repeat(levels, train_steps)

S = np.array([[1., 1., 1., 1.],
              [1., 1., 0., 0.],
              [0., 0., 1., 1.],
              [1., 0., 0., 0.],
              [0., 1., 0., 0.],
              [0., 0., 1., 0.],
              [0., 0., 0., 1.]])

S_dict = {col: S[:, i] for i, col in enumerate(levels[3:])}
S_df = pd.DataFrame(S_dict, index=levels)

ds = pd.date_range(start='2018-03-31', periods=train_steps, freq='Q').tolist() * num_levels
# Create Y_df
y_lists = [S @ np.random.uniform(low=100, high=500, size=4) for i in range(train_steps)]
y = [elem for tup in zip(*y_lists) for elem in tup]
Y_df = pd.DataFrame({'unique_id': unique_ids, 'ds': ds, 'y': y})
Y_df = sort_df_hier(Y_df, S_df)

hint_dataset, *_ = TimeSeriesDataset.from_df(df=Y_df)

In [ ]:
%%capture
# Perform a simple hyperparameter optimization with 
# NHITS and then reconcile with HINT
from neuralforecast.losses.pytorch import GMM, sCRPS

base_config = dict(max_steps=1, val_check_steps=1, input_size=8)
base_model = AutoNHITS(h=4, loss=GMM(n_components=2, quantiles=quantiles), 
                       config=base_config, num_samples=1, cpus=1)
model = HINT(h=4, S=S_df.values,
             model=base_model,  reconciliation='MinTraceOLS')

model.fit(dataset=dataset)
y_hat = model.predict(dataset=hint_dataset)

# Perform a conjunct hyperparameter optimization with 
# NHITS + HINT reconciliation configurations
nhits_config = {
       "learning_rate": tune.choice([1e-3]),                                     # Initial Learning rate
       "max_steps": tune.choice([1]),                                            # Number of SGD steps
       "val_check_steps": tune.choice([1]),                                      # Number of steps between validation
       "input_size": tune.choice([5 * 12]),                                      # input_size = multiplier * horizon
       "batch_size": tune.choice([7]),                                           # Number of series in windows
       "windows_batch_size": tune.choice([256]),                                 # Number of windows in batch
       "n_pool_kernel_size": tune.choice([[2, 2, 2], [16, 8, 1]]),               # MaxPool's Kernelsize
       "n_freq_downsample": tune.choice([[168, 24, 1], [24, 12, 1], [1, 1, 1]]), # Interpolation expressivity ratios
       "activation": tune.choice(['ReLU']),                                      # Type of non-linear activation
       "n_blocks":  tune.choice([[1, 1, 1]]),                                    # Blocks per each 3 stacks
       "mlp_units":  tune.choice([[[512, 512], [512, 512], [512, 512]]]),        # 2 512-Layers per block for each stack
       "interpolation_mode": tune.choice(['linear']),                            # Type of multi-step interpolation
       "random_seed": tune.randint(1, 10),
       "reconciliation": tune.choice(['BottomUp', 'MinTraceOLS', 'MinTraceWLS'])
    }
model = AutoHINT(h=4, S=S_df.values,
                 cls_model=NHITS,
                 config=nhits_config,
                 loss=GMM(n_components=2, level=[80, 90]),
                 valid_loss=sCRPS(level=[80, 90]),
                 num_samples=1, cpus=1)
model.fit(dataset=dataset)
y_hat = model.predict(dataset=hint_dataset)

In [ ]:
#| hide
# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoHINT) 

In [ ]:
#| export
class AutoTSMixer(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4],
        "h": None,
        "n_series": None,
        "n_block": tune.choice([1, 2, 4, 6, 8]),
        "learning_rate": tune.loguniform(1e-4, 1e-2),
        "ff_dim": tune.choice([32, 64, 128]),
        "scaler_type": tune.choice(['identity', 'robust', 'standard']),
        "max_steps": tune.choice([500, 1000, 2000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "dropout": tune.uniform(0.0, 0.99),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 n_series,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend, n_series=n_series)                 

        # Always use n_series from parameters, raise exception with Optuna because we can't enforce it
        if backend == 'ray':
            config['n_series'] = n_series
        elif backend == 'optuna':
            mock_trial = MockTrial()
            if ('n_series' in config(mock_trial) and config(mock_trial)['n_series'] != n_series) or ('n_series' not in config(mock_trial)):
                raise Exception(f"config needs 'n_series': {n_series}")

        super(AutoTSMixer, self).__init__(
              cls_model=TSMixer, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,            
        )


    @classmethod
    def get_default_config(cls, h, backend, n_series):
        config = cls.default_config.copy()        
        config['input_size'] = tune.choice([h * x \
                        for x in config["input_size_multiplier"]])

        # Rolling windows with step_size=1 or step_size=h
        # See `BaseWindows` and `BaseRNN`'s create_windows
        config['step_size'] = tune.choice([1, h])
        del config["input_size_multiplier"]
        if backend == 'optuna':
            # Always use n_series from parameters
            config['n_series'] = n_series
            config = cls._ray_config_to_optuna(config)           

        return config

In [ ]:
show_doc(AutoTSMixer, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoTSMixer.default_config
config = dict(max_steps=1, val_check_steps=1, input_size=12)
model = AutoTSMixer(h=12, n_series=1, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoTSMixer(h=12, n_series=1, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12
assert model.config(MockTrial())['n_series'] == 1

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoTSMixer, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoTSMixer.get_default_config(h=12, backend='optuna', n_series=1)
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 1, 'val_check_steps': 1, 'input_size': 12})
    return config

model = AutoTSMixer(h=12, n_series=1, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoTSMixer.get_default_config(h=12, backend='ray', n_series=1)
my_config['max_steps'] = 1
my_config['val_check_steps'] = 1
my_config['input_size'] = 12
model = AutoTSMixer(h=12, n_series=1, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

In [ ]:
#| export
class AutoTSMixerx(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4],
        "h": None,
        "n_series": None,
        "n_block": tune.choice([1, 2, 4, 6, 8]),
        "learning_rate": tune.loguniform(1e-4, 1e-2),
        "ff_dim": tune.choice([32, 64, 128]),
        "scaler_type": tune.choice(['identity', 'robust', 'standard']),
        "max_steps": tune.choice([500, 1000, 2000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "dropout": tune.uniform(0.0, 0.99),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 n_series,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend, n_series=n_series)         

        # Always use n_series from parameters, raise exception with Optuna because we can't enforce it
        if backend == 'ray':
            config['n_series'] = n_series
        elif backend == 'optuna':
            mock_trial = MockTrial()
            if ('n_series' in config(mock_trial) and config(mock_trial)['n_series'] != n_series) or ('n_series' not in config(mock_trial)):
                raise Exception(f"config needs 'n_series': {n_series}")            

        super(AutoTSMixerx, self).__init__(
              cls_model=TSMixerx, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,            
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series):
        config = cls.default_config.copy()        
        config['input_size'] = tune.choice([h * x \
                        for x in config["input_size_multiplier"]])

        # Rolling windows with step_size=1 or step_size=h
        # See `BaseWindows` and `BaseRNN`'s create_windows
        config['step_size'] = tune.choice([1, h])
        del config["input_size_multiplier"]
        if backend == 'optuna':
            # Always use n_series from parameters
            config['n_series'] = n_series
            config = cls._ray_config_to_optuna(config)           

        return config

In [ ]:
show_doc(AutoTSMixerx, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoTSMixerx.default_config
config = dict(max_steps=1, val_check_steps=1, input_size=12)
model = AutoTSMixerx(h=12, n_series=1, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoTSMixerx(h=12, n_series=1, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12
assert model.config(MockTrial())['n_series'] == 1

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoTSMixerx, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoTSMixerx.get_default_config(h=12, backend='optuna', n_series=1)
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 1, 'val_check_steps': 1, 'input_size': 12})
    return config

model = AutoTSMixerx(h=12, n_series=1, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoTSMixerx.get_default_config(h=12, backend='ray', n_series=1)
my_config['max_steps'] = 1
my_config['val_check_steps'] = 1
my_config['input_size'] = 12
model = AutoTSMixerx(h=12, n_series=1, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)


In [ ]:
#| export
class AutoMLPMultivariate(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4, 5],
        "h": None,
        "n_series": None,
        "hidden_size": tune.choice( [256, 512, 1024] ),
        "num_layers": tune.randint(2, 6),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice([None, 'robust', 'standard']),
        "max_steps": tune.choice([500, 1000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 n_series,
                 loss=MAE(),
                 valid_loss=None,     
                 config=None,
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None):

        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend, n_series=n_series)         

        # Always use n_series from parameters, raise exception with Optuna because we can't enforce it
        if backend == 'ray':
            config['n_series'] = n_series
        elif backend == 'optuna':
            mock_trial = MockTrial()
            if ('n_series' in config(mock_trial) and config(mock_trial)['n_series'] != n_series) or ('n_series' not in config(mock_trial)):
                raise Exception(f"config needs 'n_series': {n_series}")  

        super(AutoMLPMultivariate, self).__init__(
              cls_model=MLPMultivariate,
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config, 
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,            
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series):
        config = cls.default_config.copy()        
        config['input_size'] = tune.choice([h * x \
                        for x in config["input_size_multiplier"]])

        # Rolling windows with step_size=1 or step_size=h
        # See `BaseWindows` and `BaseRNN`'s create_windows
        config['step_size'] = tune.choice([1, h])
        del config["input_size_multiplier"]
        if backend == 'optuna':
            # Always use n_series from parameters
            config['n_series'] = n_series
            config = cls._ray_config_to_optuna(config)           

        return config

In [ ]:
show_doc(AutoMLPMultivariate, title_level=3)

In [ ]:
%%capture
# Use your own config or AutoTSMixerx.default_config
config = dict(max_steps=1, val_check_steps=1, input_size=12)
model = AutoMLPMultivariate(h=12, n_series=1, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoMLPMultivariate(h=12, n_series=1, config=None, backend='optuna')

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12
assert model.config(MockTrial())['n_series'] == 1

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoMLPMultivariate, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoMLPMultivariate.get_default_config(h=12, backend='optuna', n_series=1)
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 1, 'val_check_steps': 1, 'input_size': 12})
    return config

model = AutoMLPMultivariate(h=12, n_series=1, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoMLPMultivariate.get_default_config(h=12, backend='ray', n_series=1)
my_config['max_steps'] = 1
my_config['val_check_steps'] = 1
my_config['input_size'] = 12
model = AutoMLPMultivariate(h=12, n_series=1, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

In [ ]:
#| export
class AutoSOFTS(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4, 5],
        "h": None,
        "n_series": None,
        "hidden_size": tune.choice([64, 128, 256, 512]),
        "d_core": tune.choice([64, 128, 256, 512]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice([None, 'robust', 'standard', 'identity']),
        "max_steps": tune.choice([500, 1000, 2000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 n_series,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None,
                 backend='ray',
                 callbacks=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.get_default_config(h=h, backend=backend, n_series=n_series)                 

        # Always use n_series from parameters, raise exception with Optuna because we can't enforce it
        if backend == 'ray':
            config['n_series'] = n_series
        elif backend == 'optuna':
            mock_trial = MockTrial()
            if ('n_series' in config(mock_trial) and config(mock_trial)['n_series'] != n_series) or ('n_series' not in config(mock_trial)):
                raise Exception(f"config needs 'n_series': {n_series}")           

        super(AutoSOFTS, self).__init__(
              cls_model=SOFTS, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
              backend=backend,
              callbacks=callbacks,            
        )

    @classmethod
    def get_default_config(cls, h, backend, n_series):
        config = cls.default_config.copy()        
        config['input_size'] = tune.choice([h * x \
                        for x in config["input_size_multiplier"]])

        # Rolling windows with step_size=1 or step_size=h
        # See `BaseWindows` and `BaseRNN`'s create_windows
        config['step_size'] = tune.choice([1, h])
        del config["input_size_multiplier"]
        if backend == 'optuna':
            # Always use n_series from parameters
            config['n_series'] = n_series
            config = cls._ray_config_to_optuna(config)           

        return config         

In [ ]:
show_doc(AutoSOFTS, title_level=3)

---

[source](https://github.com/Nixtla/neuralforecast/blob/main/neuralforecast/auto.py#L2085){target="_blank" style="float:right; font-size:smaller"}

### AutoSOFTS

>      AutoSOFTS (h, n_series, loss=MAE(), valid_loss=None, config=None,
>                 search_alg=<ray.tune.search.basic_variant.BasicVariantGenerato
>                 r object at 0x32ce6d180>, num_samples=10,
>                 refit_with_val=False, cpus=11, gpus=0, verbose=False,
>                 alias=None, backend='ray', callbacks=None)

Class for Automatic Hyperparameter Optimization, it builds on top of `ray` to
give access to a wide variety of hyperparameter optimization tools ranging
from classic grid search, to Bayesian optimization and HyperBand algorithm.

The validation loss to be optimized is defined by the `config['loss']` dictionary
value, the config also contains the rest of the hyperparameter search space.

It is important to note that the success of this hyperparameter optimization
heavily relies on a strong correlation between the validation and test periods.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| h | int |  | Forecast horizon |
| n_series |  |  |  |
| loss | MAE | MAE() | Instantiated train loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html). |
| valid_loss | NoneType | None | Instantiated valid loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html). |
| config | NoneType | None | Dictionary with ray.tune defined search space or function that takes an optuna trial and returns a configuration dict. |
| search_alg | BasicVariantGenerator | <ray.tune.search.basic_variant.BasicVariantGenerator object> | For ray see https://docs.ray.io/en/latest/tune/api_docs/suggestion.html<br>For optuna see https://optuna.readthedocs.io/en/stable/reference/samplers/index.html. |
| num_samples | int | 10 | Number of hyperparameter optimization steps/samples. |
| refit_with_val | bool | False | Refit of best model should preserve val_size. |
| cpus | int | 11 | Number of cpus to use during optimization. Only used with ray tune. |
| gpus | int | 0 | Number of gpus to use during optimization, default all available. Only used with ray tune. |
| verbose | bool | False | Track progress. |
| alias | NoneType | None | Custom name of the model. |
| backend | str | ray | Backend to use for searching the hyperparameter space, can be either 'ray' or 'optuna'. |
| callbacks | NoneType | None | List of functions to call during the optimization process.<br>ray reference: https://docs.ray.io/en/latest/tune/tutorials/tune-metrics.html<br>optuna reference: https://optuna.readthedocs.io/en/stable/tutorial/20_recipes/007_optuna_callback.html |

In [ ]:
%%capture
# Use your own config or AutoSOFTS.default_config
config = dict(max_steps=1, val_check_steps=1, input_size=12, hidden_size=16)
model = AutoSOFTS(h=12, n_series=1, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Optuna
model = AutoSOFTS(h=12, n_series=1, config=None, backend='optuna')

2024-05-31 15:06:51,623	INFO worker.py:1749 -- Started a local Ray instance.
2024-05-31 15:06:52,217	INFO tune.py:263 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.
2024-05-31 15:06:52,219	INFO tune.py:624 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
2024-05-31 15:06:54,633	ERROR tune_controller.py:1332 -- Trial task failed for trial _train_tune_eb814_00000
Traceback (most recent call last):
  File "/Users/marcopeix/miniconda3/envs/neuralforecast/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/Users/marcopeix/miniconda3/envs/neuralforecast/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwarg

In [ ]:
#| hide
# Check Optuna
assert model.config(MockTrial())['h'] == 12

# Unit test to test that Auto* model contains all required arguments from BaseAuto
test_args(AutoSOFTS, exclude_args=['cls_model']) 

# Unit test for situation: Optuna with updated default config
my_config = AutoSOFTS.get_default_config(h=12, n_series=1, backend='optuna')
def my_config_new(trial):
    config = {**my_config(trial)}
    config.update({'max_steps': 1, 'val_check_steps': 1, 'input_size': 12, 'hidden_size': 16})
    return config

model = AutoSOFTS(h=12, n_series=1, config=my_config_new, backend='optuna', num_samples=1, cpus=1)
model.fit(dataset=dataset)

# Unit test for situation: Ray with updated default config
my_config = AutoSOFTS.get_default_config(h=12, n_series=1, backend='ray')
my_config['max_steps'] = 1
my_config['val_check_steps'] = 1
my_config['input_size'] = 12
my_config['hidden_size'] = 16
model = AutoSOFTS(h=12, n_series=1, config=my_config, backend='ray', num_samples=1, cpus=1)
model.fit(dataset=dataset)

[I 2024-05-31 15:07:11,775] A new study created in memory with name: no-name-9172b813-168d-48bf-9ae8-6d9dc8603e2b
Seed set to 17
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1)` was configured so validation will run after every batch.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

[W 2024-05-31 15:07:11,926] Trial 0 failed with parameters: {'hidden_size': 256, 'd_core': 256, 'learning_rate': 0.0003957532292586766, 'scaler_type': 'robust', 'max_steps': 2000, 'batch_size': 32, 'random_seed': 17, 'input_size': 24, 'step_size': 1} because of the following error: NotImplementedError("The operator 'aten::nanmedian.dim_values' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.").
Traceback (most recent call last):
  File "/Users/marcopeix/miniconda3/envs/neuralforecast/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/Users/marcopeix/miniconda3/envs/neuralfore

NotImplementedError: The operator 'aten::nanmedian.dim_values' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

# TESTS

In [ ]:
#| hide
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengersDF as Y_df

In [ ]:
#| hide
# Split train/test and declare time series dataset
Y_train_df = Y_df[Y_df.ds<='1959-12-31'] # 132 train
Y_test_df = Y_df[Y_df.ds>'1959-12-31']   # 12 test
dataset, *_ = TimeSeriesDataset.from_df(Y_train_df)

config = dict(max_steps=1, val_check_steps=1, input_size=12)
model = AutoNHITS(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

In [ ]:
#| hide
## TESTS
nhits_config = {
       "learning_rate": tune.choice([1e-3]),                                     # Initial Learning rate
       "max_steps": tune.choice([1]),                                            # Number of SGD steps
       "val_check_steps": tune.choice([1]),                                      # Number of steps between validation
       "input_size": tune.choice([5 * 12]),                                      # input_size = multiplier * horizon
       "batch_size": tune.choice([7]),                                           # Number of series in windows
       "windows_batch_size": tune.choice([256]),                                 # Number of windows in batch
       "n_pool_kernel_size": tune.choice([[2, 2, 2], [16, 8, 1]]),               # MaxPool's Kernelsize
       "n_freq_downsample": tune.choice([[168, 24, 1], [24, 12, 1], [1, 1, 1]]), # Interpolation expressivity ratios
       "activation": tune.choice(['ReLU']),                                      # Type of non-linear activation
       "n_blocks":  tune.choice([[1, 1, 1]]),                                    # Blocks per each 3 stacks
       "mlp_units":  tune.choice([[[512, 512], [512, 512], [512, 512]]]),        # 2 512-Layers per block for each stack
       "interpolation_mode": tune.choice(['linear']),                            # Type of multi-step interpolation
       "random_seed": tune.randint(1, 10),
    }

model = AutoNHITS(h=12, loss=MAE(), valid_loss=MSE(), config=nhits_config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Test equality
test_eq(str(type(model.valid_loss)), "<class 'neuralforecast.losses.pytorch.MSE'>")

In [ ]:
#| hide
from neuralforecast.losses.pytorch import GMM, sCRPS

In [ ]:
#| hide
## TODO: Add unit tests for interactions between loss/valid_loss types
## TODO: Unit tests (2 types of networks x 2 types of loss x 2 types of valid loss)
## Checking if base recurrent methods run point valid_loss correctly
tcn_config = {
       "learning_rate": tune.choice([1e-3]),                                     # Initial Learning rate
       "max_steps": tune.choice([1]),                                            # Number of SGD steps
       "val_check_steps": tune.choice([1]),                                      # Number of steps between validation
       "input_size": tune.choice([5 * 12]),                                      # input_size = multiplier * horizon
       "batch_size": tune.choice([7]),                                           # Number of series in windows
       "random_seed": tune.randint(1, 10),
    }

model = AutoTCN(h=12, 
                loss=MAE(), 
                valid_loss=MSE(), 
                config=tcn_config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

## Checking if base recurrent methods run quantile valid_loss correctly
model = AutoTCN(h=12, 
                loss=GMM(n_components=2, level=[80, 90]),
                valid_loss=sCRPS(level=[80, 90]),
                config=tcn_config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)